In [1]:
# Simple RAG System for Ancient Greece Q&A

import os
import re
from pathlib import Path
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_milvus import Milvus
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pymilvus import connections

In [ ]:
# Configuration
DATA_DIR = "/mnt/d/M-Bank/Ancient Greece QA Chatbot/ancient_greece_data_omnispay"
CHUNK_SIZE = 800
CHUNK_OVERLAP = 100
RETRIEVAL_K = 4
EMBEDDING_MODEL = "BAAI/bge-base-en-v1.5"
LLM_MODEL = "llama3.2"
LLM_BASE_URL = "http://localhost:11434"
MILVUS_HOST = "127.0.0.1"
MILVUS_PORT = "19530"
COLLECTION_NAME = "BankingRAGCollection_Omnispay"

In [5]:
# Step 1: Load Documents
print("Loading documents...")
loader = DirectoryLoader(
    DATA_DIR,
    glob='**/*.txt',
    loader_cls=TextLoader
)
docs = loader.load()
print(f"Loaded {len(docs)} documents")

Loading documents...
Loaded 1 documents


In [6]:
# Step 2: Split Documents into Chunks
print("Splitting documents into chunks...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", ". ", " "]
)
chunks = text_splitter.split_documents(docs)
print(f"Created {len(chunks)} chunks")

Splitting documents into chunks...
Created 71 chunks


In [7]:
# Step 3: Create Embeddings and Vector Store
print("Creating embeddings and vector store...")
embedding = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

# Connect to Milvus
print("Connecting to Milvus...")
connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)

# Create Milvus vector store
vectorstore = Milvus.from_documents(
    documents=chunks,
    embedding=embedding,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT},
    collection_name=COLLECTION_NAME,
    drop_old=True  # Remove existing collection if it exists
)
print("Milvus vector store created!")

Creating embeddings and vector store...


/mnt/d/M-Bank/banking_chatbot_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connecting to Milvus...


2025-07-07 07:44:21,153 [DEBUG][_create_connection]: Created new connection using: 303385e227124a138d90615a9bf12fc2 (async_milvus_client.py:599)


Milvus vector store created!


In [30]:
# Step 4: Initialize LLM
print("Connecting to LLM...")
llm = ChatOllama(
    model=LLM_MODEL,
    base_url=LLM_BASE_URL,
    temperature=0.1
)

Connecting to LLM...


In [31]:
# Step 5: Create Simple Retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": RETRIEVAL_K}
)

In [32]:
# Step 6: Create Prompt Template
prompt_template = """You are an expert on Finance . Answer the question using the provided context.

Context:
{context}

Question: {question}

Answer:"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)

In [33]:
# Step 7: Format Context Function
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [34]:
# Step 8: Build RAG Chain
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [35]:
# Step 9: Question Answering Function
def ask_question(question):
    """Ask a question and get an answer"""
    print(f"\nQuestion: {question}")
    print("-" * 50)
    
    try:
        answer = rag_chain.invoke(question)
        print(f"Answer: {answer}")
        return answer
    except Exception as e:
        print(f"Error: {e}")
        return None

In [39]:
# Step 10: Test the System
print("\n" + "="*60)
print("RAG SYSTEM READY!")
print("="*60)

# Test questions
test_questions = [
    "what is interest rate on Savings Account"
]

# Run tests
print("\nTesting the system...")
for question in test_questions:
    ask_question(question)
    print()



RAG SYSTEM READY!

Testing the system...

Question: what is interest rate on Savings Account
--------------------------------------------------
Answer: The interest rate on a Savings Account is 3.5% p.a.



In [ ]:
# # Interactive mode function
# def interactive_mode():
#     """Run interactive Q&A session"""
#     print("\nInteractive Mode - Type 'quit' to exit")
#     print("-" * 40)
    
#     while True:
#         question = input("\nYour question: ").strip()
#         if question.lower() in ['quit', 'exit', 'q']:
#             print("Goodbye!")
#             break
#         if question:
#             ask_question(question)

In [ ]:
# # Save/Load functions for Milvus (optional)
# def get_existing_vectorstore():
#     """Load existing Milvus collection if it exists"""
#     try:
#         connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)
        
#         embedding = HuggingFaceEmbeddings(
#             model_name=EMBEDDING_MODEL,
#             model_kwargs={"device": "cpu"},
#             encode_kwargs={"normalize_embeddings": True}
#         )
        
#         vectorstore = Milvus(
#             embedding_function=embedding,
#             connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT},
#             collection_name=COLLECTION_NAME
#         )
        
#         print("Loaded existing Milvus collection!")
#         return vectorstore
        
#     except Exception as e:
#         print(f"Could not load existing collection: {e}")
#         return None